In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [3]:
!pip install torch

In [4]:
from torch.utils.data import Dataset, DataLoader

In [5]:
class MNISTData(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.data[idx]

In [6]:
class MNISTTrainData(MNISTData):
    def __getitem__(self, idx):
        X = self.data[idx][1:] / 255.0    # Normalisation
        y = self.data[idx][0]
        return X,y

In [7]:
class MNISTTestData(MNISTData):
    def __getitem__(self, idx):
        X = self.data[idx] / 255.0     # Normalisation
        return X

In [8]:
import pandas as pd

In [9]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [10]:
print(train_data.shape)
print(test_data.shape)

(42000, 785)
(28000, 784)


In [11]:
train_data = MNISTTrainData(train_data.values)
test_data = MNISTTestData(test_data.values)

In [12]:
batch_size = 512
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=batch_size, shuffle=False)

In [13]:
import torch
import torch.nn as nn

In [18]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )
        
    def forward(self, x):
        output = self.mlp(x)
        return output

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
learning_rate = 0.01

In [21]:
model = MLP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [22]:
print(model)

MLP(
  (mlp): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [23]:
epochs = 20

model.train()

for epoch in range(epochs):
    losses = []
    
    for batch_num, input_data in enumerate(train_loader):
        optimizer.zero_grad()
        
        x,y = input_data
        x = x.to(device).float()
        y = y.to(device)
        
        output = model(x)
        
        loss = criterion(output, y)
        loss.backward()
        losses.append(loss.item())
        
        optimizer.step()
        
        if batch_num % 40 == 0:
            print('\tEpoch %d | Batch %d | Loss %6.2f' % (epoch, batch_num, loss.item()))
    print('Epoch %d | Loss %6.2f' % (epoch, sum(losses)/len(losses)))

	Epoch 0 | Batch 0 | Loss   2.31
	Epoch 0 | Batch 40 | Loss   0.22
	Epoch 0 | Batch 80 | Loss   0.21
Epoch 0 | Loss   0.38
	Epoch 1 | Batch 0 | Loss   0.23
	Epoch 1 | Batch 40 | Loss   0.16
	Epoch 1 | Batch 80 | Loss   0.12
Epoch 1 | Loss   0.15
	Epoch 2 | Batch 0 | Loss   0.13
	Epoch 2 | Batch 40 | Loss   0.11
	Epoch 2 | Batch 80 | Loss   0.09
Epoch 2 | Loss   0.11
	Epoch 3 | Batch 0 | Loss   0.07
	Epoch 3 | Batch 40 | Loss   0.07
	Epoch 3 | Batch 80 | Loss   0.05
Epoch 3 | Loss   0.07
	Epoch 4 | Batch 0 | Loss   0.04
	Epoch 4 | Batch 40 | Loss   0.06
	Epoch 4 | Batch 80 | Loss   0.04
Epoch 4 | Loss   0.06
	Epoch 5 | Batch 0 | Loss   0.03
	Epoch 5 | Batch 40 | Loss   0.04
	Epoch 5 | Batch 80 | Loss   0.01
Epoch 5 | Loss   0.04
	Epoch 6 | Batch 0 | Loss   0.04
	Epoch 6 | Batch 40 | Loss   0.02
	Epoch 6 | Batch 80 | Loss   0.04
Epoch 6 | Loss   0.04
	Epoch 7 | Batch 0 | Loss   0.02
	Epoch 7 | Batch 40 | Loss   0.02
	Epoch 7 | Batch 80 | Loss   0.03
Epoch 7 | Loss   0.03
	Epoch 8 | Batch

In [24]:
import csv
model.eval()

results = []

with torch.no_grad():
    for batch_number, x in enumerate(test_loader):
        x = x.to(device).float()

        output = model(x).argmax(dim=1)
        for out in output:
            results.append(out.item())

In [ ]:
len(results)

In [25]:
submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [26]:
submission['Label'] = results

In [27]:
submission.to_csv("/kaggle/working/submission.csv", index = False)